In [1]:
import os
import pandas as pd
from pathlib import Path
from typing import Iterable, Tuple

# ========= 用户配置 =========
CSV_PATH      = Path("/Users/hezizhao/Downloads/jupython-workspace/FH2/fh2_desils.csv")  # 修改为你的表格路径
SAVE_DIR      = Path("./data/fitscutout")         # 下载保存目录
NPIXEL        = 21                                 # cutout 像素尺寸（方形：NPIXEL x NPIXEL）
ROUND_DIGITS  = 5                                  # 文件名保留小数位（避免覆盖）
LAYER         = "ls-dr10"                    # Legacy Surveys 图层
USE_SUBIMAGE  = True                               # 是否加 &subimage 参数
DRY_RUN       = False                              # True=只打印命令不执行


# ========= 下载函数（基于 wget，与用户给出版本兼容） =========
def Download_fitsimg_subimage(ra: float,
                              dec: float,
                              npixel: int = NPIXEL,
                              save_dir: Path = SAVE_DIR,
                              layer: str = LAYER,
                              round_digits: int = ROUND_DIGITS,
                              use_subimage: bool = USE_SUBIMAGE,
                              dry_run: bool = DRY_RUN) -> Path:
    """
    下载 Legacy Survey cutout 的简化包装。
    返回保存的 FITS 路径（Path 对象）。
    """
    save_dir.mkdir(parents=True, exist_ok=True)

    # 文件名（避免空格、科学计数法；保留 round_digits 位小数）
    savename = f"ra_{ra:.{round_digits}f}_dec_{dec:.{round_digits}f}-subimage.fits"
    savepath = save_dir / savename

    # 若已存在，直接跳过
    if savepath.exists():
        print(f"[skip] exists: {savepath}")
        return savepath

    # 组装 URL
    # 示例: https://www.legacysurvey.org/viewer/cutout.fits?ra=19.8023&dec=10.3749&layer=ls-dr10-resid&size=100&subimage
    url = (
        f"https://www.legacysurvey.org/viewer/cutout.fits"
        f"?ra={ra}&dec={dec}&layer={layer}&size={npixel}"
    )
    if use_subimage:
        url += "&subimage"

    # wget 命令：-O 指定输出文件名可避免再 mv
    cmd = f"wget -q -O '{savepath}' '{url}'"

    print(cmd)
    if not dry_run:
        status = os.system(cmd)
        if status != 0:
            print(f"[warn] wget status {status} for RA={ra}, DEC={dec}")
    return savepath


# ========= 从表格读取并去重 RA/Dec =========
def load_unique_coords(csv_path: Path,
                       ra_col: str = "ra",
                       dec_col: str = "dec") -> Iterable[Tuple[float, float]]:
    """
    读表并返回唯一 (ra, dec) 列表（去重后顺序按首次出现保持）。
    """
    df = pd.read_csv(csv_path)

    # 防御：确保列存在；若大小写不一致自动查找
    cols_lower = {c.lower(): c for c in df.columns}
    if ra_col not in cols_lower:
        raise KeyError(f"RA column '{ra_col}' not found in CSV (available: {list(df.columns)})")
    if dec_col not in cols_lower:
        raise KeyError(f"DEC column '{dec_col}' not found in CSV (available: {list(df.columns)})")

    ra_c = cols_lower[ra_col]
    dec_c = cols_lower[dec_col]

    # 去重（保持原顺序）
    dedup = df[[ra_c, dec_c]].drop_duplicates()

    # 转 float
    dedup = dedup.astype(float)

    return list(zip(dedup[ra_c].to_numpy(), dedup[dec_c].to_numpy()))


# ========= 主程序 =========
def batch_download(csv_path: Path = CSV_PATH):
    coords = load_unique_coords(csv_path)
    print(f"Total rows in table: {len(pd.read_csv(csv_path))}")
    print(f"Unique (ra, dec) pairs: {len(coords)}")
    print("Beginning downloads...\n")

    for i, (ra, dec) in enumerate(coords, start=1):
        print(f"[{i}/{len(coords)}] RA={ra} DEC={dec}")
        Download_fitsimg_subimage(ra, dec)

    print("\nDone.")


# 如果你想在导入后立即运行，取消下面注释：


In [3]:
batch_download()

Total rows in table: 68
Unique (ra, dec) pairs: 30
Beginning downloads...

[1/30] RA=179.093353 DEC=21.987776
[skip] exists: data/fitscutout/ra_179.09335_dec_21.98778-subimage.fits
[2/30] RA=244.706042 DEC=6.476774
[skip] exists: data/fitscutout/ra_244.70604_dec_6.47677-subimage.fits
[3/30] RA=100.652938 DEC=56.286423
[skip] exists: data/fitscutout/ra_100.65294_dec_56.28642-subimage.fits
[4/30] RA=4.946581 DEC=1.500982
[skip] exists: data/fitscutout/ra_4.94658_dec_1.50098-subimage.fits
[5/30] RA=65.581298 DEC=-4.79716
wget -q -O 'data/fitscutout/ra_65.58130_dec_-4.79716-subimage.fits' 'https://www.legacysurvey.org/viewer/cutout.fits?ra=65.581298&dec=-4.79716&layer=ls-dr10&size=21&subimage'
[6/30] RA=9.316034 DEC=20.973797
[skip] exists: data/fitscutout/ra_9.31603_dec_20.97380-subimage.fits
[7/30] RA=234.254361 DEC=32.39078
[skip] exists: data/fitscutout/ra_234.25436_dec_32.39078-subimage.fits
[8/30] RA=246.258188 DEC=43.15886
[skip] exists: data/fitscutout/ra_246.25819_dec_43.15886-sub